# Cross validation and the need for self evaluation
In order to understand how affective our method is in producing predictions that minimize the cost function (WRMSSE), we must develop cross validation strategies in which we make predictions on different validation datasets. In order to create different validation sets, we must have a tool that can accurately score our predictions in accordance with the WRMSSE. This is particularly difficult because the WRMSSE is based on weights and scaling factors that must be recalculated for every validation set. In this notebook, we will develop the tools to accurately implement the WRMSSE for any validation period with 28 days of know data. The last cell will be a WRMSSE object and helper functions that can easily used. 

# WRMSSE
$$
WRMSSE = \sum_{i=1}^{42,840} \left(W_i \times \sqrt{\frac{\sum_{j=1}^{28}{(D_j)^2}}{S_i}}\right)
$$
* W_i: the weight of the ith series 
* S_i: the scaling factor of the ith series 
* D_j: The difference between sales and predicted sales for the ith series on day j

I prefer to look at the equation like this: 
$$
WRMSSE = \sum_{i=1}^{42,840} \frac{W_i}{\sqrt{S_i}} \times \sqrt{\sum_{j=1}^{28}{(D_j)^2}}
$$
To build a WRMSSE scoring object, we will need to create tools that can apply this caclulation as efficiently as possible. We will develop a sparse aggregation matrix, created with a one-hot-encoding style, that serves to compute the aggregations for all 42840 series from the bottme level 30490 series. After the aggregation matrix, we will develop methods to compute the weights W and the scaling factor S for all series. We will then combine our tools to create a WRMSSE object, capable of scoring predictions for any 28 validation period of known data.  

In [ ]:
import pandas as pd 
import numpy as np 
from scipy.sparse import csr_matrix

In [ ]:
################## Variables ####################
# We will work through an example of calculating 
# the WRMSSE by level, and overall. Then we will 
# 
START_TEST = 1914 
END_TRAIN = START_TEST - 1 # last training day

In [ ]:
############# Load data ###################
DATA_PATH = '/kaggle/input/m5-forecasting-accuracy/'
train_df = pd.read_csv(f'{DATA_PATH}sales_train_validation.csv')
prices_df = pd.read_csv(f'{DATA_PATH}sell_prices.csv')
cal_df = pd.read_csv(f'{DATA_PATH}calendar.csv')

In [ ]:
%%time
#################### Dummy matrix #####################
# We know we can compute all the aggregated series by 
# using matrix multiplication with the correctly 
# designed "rollup" matrix. Our daily sales have the 
# shape (number_items, prediction_horizon). Our rollup
# matrix will need to have the shape 
# (number_series, number_items) so that we can execute 
# the matrix multiplication rollup x sales.

# We need a list of the aggregating features that
# will align with our weights and scales so that 
# our matrices will match up. Level 1 does not need
# a column to group by.

# For each sereis of each level of the WRMSSE, we will 
# use pandas get_dummies function on the corresponding
# column or columns. 
dummy_matrices = [
    pd.DataFrame({'all': np.ones((30490,)).astype('int8')}, index=train_df.index).T, 
    pd.get_dummies(train_df.state_id, dtype=np.int8).T,                                             
    pd.get_dummies(train_df.store_id, dtype=np.int8).T,
    pd.get_dummies(train_df.cat_id, dtype=np.int8).T,
    pd.get_dummies(train_df.dept_id, dtype=np.int8).T,
    pd.get_dummies(train_df.state_id + '_' + train_df.cat_id, dtype=np.int8).T,
    pd.get_dummies(train_df.state_id + '_' + train_df.dept_id, dtype=np.int8).T,
    pd.get_dummies(train_df.store_id + '_' + train_df.cat_id, dtype=np.int8).T,
    pd.get_dummies(train_df.store_id + '_' + train_df.dept_id, dtype=np.int8).T,
    pd.get_dummies(train_df.item_id, dtype=np.int8).T,
    pd.get_dummies(train_df.item_id + '_' + train_df.state_id, dtype=np.int8).T,
    pd.get_dummies(train_df.item_id + '_' + train_df.store_id, dtype=np.int8).T
]

# Take the transpose to correctly orient the matrix
rollup_matrix = pd.concat(dummy_matrices, keys=range(1,13), names=['Level', 'id'])

# Save the index for later use 
rollup_index = rollup_matrix.index

# Sparse format will save space and calculation time
rollup_matrix_csr = csr_matrix(rollup_matrix)

Note that the weight of each series will be computed based on the last 28 observations of the training
sample of the dataset, i.e., the cumulative actual dollar sales that each series displayed in that particular
period (sum of units sold multiplied by their respective price).

In [ ]:
##################### Weights ########################
# We need to convert the sales data into dollar sales 
# data so that we can correctly weight each series. 

# To begin, we consider only the last 28 days of 
# data before START_TEST. We then put the data into 
# "long" format so we can merge the calendar 
# and price information.
d_cols = [f'd_{i}' for i in range(START_TEST - 28, START_TEST)]
df = train_df[['store_id', 'item_id'] + d_cols]
df = df.melt(id_vars=['store_id', 'item_id'],
                       var_name='d', 
                       value_name = 'sales')
df = df.merge(cal_df[['d', 'wm_yr_wk']], on='d', how='left')
df = df.merge(prices_df, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')
df['dollar_sales'] = df.sales * df.sell_price

# Now we will get the total dollar sales for each 
# item/store combination. Be sure to set sort=False
# so that our index stays in the proper order. 
# We don't need df anymore
dollar_sales = df.groupby(['store_id', 'item_id'], sort=False)['dollar_sales'].sum()
del df

# We want to build a weight, scales,
# and scaled weight columns 
# that are aligned with rollup_index. We 
# will divide dollar_sales by the total 
# dollar sales to get the weight W 
# for each series. We don't need dollar_sales anymore.
w_df = pd.DataFrame(index = rollup_index)
w_df['dollar_sales'] = rollup_matrix_csr * dollar_sales
w_df['weight'] = w_df.dollar_sales / w_df.dollar_sales[0]
del w_df['dollar_sales']

##################### Scaling factor #######################
# We also need to calculate each series scaling factor S, 
# which is the denominator in the WRMSSE cacluation. It can 
# be pulled out of the square root and combined with the 
# series weight to make a single weight W/sqrt(S),
# simplifying our calculations a bit. 

# S is the average squared difference of day to daily sales 
# for a series, excluding leading zeros, for all training 
# days leading up to START_TEST. 
df = train_df.loc[:, 'd_1':f'd_{END_TRAIN}']

# Aggregate all series, and replace leading 
# zeros with np.nan so that we can do numpy calculations 
# that will ignore the np.nan.
agg_series = rollup_matrix_csr * df.values
no_sale = np.cumsum(agg_series, axis=1) == 0
agg_series = np.where(no_sale, np.nan, agg_series)
scale = np.nanmean(np.diff(agg_series, axis=1) ** 2, axis=1)

# Now we can finish our weights and scales dataframe by 
# adding scale and scaled_weight columns. 
w_df['scale'] = scale
w_df['scaled_weight'] = w_df.weight / np.sqrt(w_df.scale)
w_df

In [ ]:
################## sample scoreing ##############
actuals = train_df.iloc[:, -28:].values
preds = np.ones((30490, 28)) 
diff = actuals - preds

np.sum(
    np.sqrt(
        np.mean((rollup_matrix_csr * diff)**2, axis=1)
           ) * w_df.scaled_weight
      ) / 12

In [ ]:
####################### Object oriented approach ######################
# We will all of our tools into functions and utilize them in a WRMSSE 
# object. 

# Imports necessary for the functions and object
import pandas as pd 
import numpy as np 
from scipy.sparse import csr_matrix

def get_rollup(train_df):
    """Gets a sparse roll up matrix for aggregation and 
    an index to align weights and scales."""
    
    # Take the transpose to correctly orient the matrix
    dummy_frames = [
        pd.DataFrame({'all': np.ones((30490,)).astype('int8')}, index=train_df.index).T, 
        pd.get_dummies(train_df.state_id, dtype=np.int8).T,                                             
        pd.get_dummies(train_df.store_id, dtype=np.int8).T,
        pd.get_dummies(train_df.cat_id, dtype=np.int8).T,
        pd.get_dummies(train_df.dept_id, dtype=np.int8).T,
        pd.get_dummies(train_df.state_id + '_' + train_df.cat_id, dtype=np.int8).T,
        pd.get_dummies(train_df.state_id + '_' + train_df.dept_id, dtype=np.int8).T,
        pd.get_dummies(train_df.store_id + '_' + train_df.cat_id, dtype=np.int8).T,
        pd.get_dummies(train_df.store_id + '_' + train_df.dept_id, dtype=np.int8).T,
        pd.get_dummies(train_df.item_id, dtype=np.int8).T,
        pd.get_dummies(train_df.item_id + '_' + train_df.state_id, dtype=np.int8).T,
        pd.get_dummies(train_df.item_id + '_' + train_df.store_id, dtype=np.int8).T
    ]

    rollup_matrix = pd.concat(dummy_frames, keys=range(1,13), names=['Level', 'id'])

    # Save the index for later use 
    rollup_index = rollup_matrix.index

    # Sparse format will save space and calculation time
    rollup_matrix_csr = csr_matrix(rollup_matrix)
    
    return rollup_matrix_csr, rollup_index



def get_w_df(train_df, cal_df, prices_df, start_test, rollup_index): 
    """Returns the weight, scale, and scaled weight of all series, 
    in a dataframe aligned with the rollup_index, created in get_rollup()"""
    
    d_cols = [f'd_{i}' for i in range(start_test - 28, start_test)]
    df = train_df[['store_id', 'item_id'] + d_cols]
    df = df.melt(id_vars=['store_id', 'item_id'],
                           var_name='d', 
                           value_name = 'sales')
    df = df.merge(cal_df[['d', 'wm_yr_wk']], on='d', how='left')
    df = df.merge(prices_df, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')
    df['dollar_sales'] = df.sales * df.sell_price

    # Now we will get the total dollar sales 
    dollar_sales = df.groupby(['store_id', 'item_id'], sort=False)['dollar_sales'].sum()
    del df

    # Build a weight, scales, and scaled weight columns 
    # that are aligned with rollup_index. 
    w_df = pd.DataFrame(index = rollup_index)
    w_df['dollar_sales'] = rollup_matrix_csr * dollar_sales
    w_df['weight'] = w_df.dollar_sales / w_df.dollar_sales[0]
    del w_df['dollar_sales']

    ##################### Scaling factor #######################
    
    df = train_df.loc[:, 'd_1':f'd_{start_test-1}']
    agg_series = rollup_matrix_csr * df.values
    no_sale = np.cumsum(agg_series, axis=1) == 0
    agg_series = np.where(no_sale, np.nan, agg_series)
    scale = np.nanmean(np.diff(agg_series, axis=1) ** 2, axis=1)

    w_df['scale'] = scale
    w_df['scaled_weight'] = w_df.weight / np.sqrt(w_df.scale)
    
    return w_df


####################### WRMSSE object ##########################
class WRMSSE():
    """An object that is capable of scoring predictions for any 
    time period, provied the necessary dataframes."""
    def __init__(self, train_df, cal_df, prices_df, start_test): 
        self.train_df = train_df
        self.cal_df = cal_df
        self.prices_df = prices_df 
        self.start_test = start_test
        self.actuals = train_df.loc[:, f'd_{self.start_test - 28}': f'd_{self.start_test - 1}'].values
        
        self.rollup_index, self.rollup_matrix_csr = get_rollup(self.train_df)
        self.w_df = get_w_df(self.train_df, self.cal_df, self.prices_df, self.start_test)
        
        self.level_scores = []
        
    def score(self, preds): 
        if type(preds) == pd.DataFrame: 
            preds = preds.values
        diff = self.actuals - preds
        res = np.sum(
                np.sqrt(
                    np.mean((self.rollup_matrix_csr * diff)**2, axis=1)
                       ) * self.w_df.scaled_weight
                  ) / 12
        return res